### Classes of Maze and Weighted Maze

In [1]:
class Maze(object):
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.walls = []
    
    def in_bounds(self, id):
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id):
        return id not in self.walls
    
    def neighbors(self, id):
        (x, y) = id
        results = [(x+1, y), (x, y-1), (x-1, y), (x, y+1)]
        results = filter(self.in_bounds, results)
        results = filter(self.passable, results)
        return results
    
class MazeWithWeights(Maze):
    def __init__(self, width, height):
        super(MazeWithWeights, self).__init__(width, height)
        self.weights = {}
    
    def cost(self, from_node, to_node):
        return self.weights.get(to_node, 1)

### Draw Maze

In [2]:
def draw_tile(graph, id, style, width):
    r = "."
    if 'number' in style and id in style['number']: r = "%d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = u"\u2192"
        if x2 == x1 - 1: r = u"\u2190"
        if y2 == y1 + 1: r = u"\u2193"
        if y2 == y1 - 1: r = u"\u2191"
    if 'start' in style and id == style['start']: r = "S"
    if 'goal' in style and id == style['goal']: r = "G"
    if 'path' in style and id in style['path']: r = "o"
    if id in graph.walls: r = u"\u254b" #u"\u2588"
    return r

def draw_grid(graph, width=2, **style):
    for y in range(graph.height):
        for x in range(graph.width):
            print"%%-%ds" % width % draw_tile(graph, (x, y), style, width),
        print "";

### Dijkstra search 

Dijkstra's algorithm is an algorithm to use a single node as the "source" node and finds shortest paths from the source to all other nodes in the graph.
https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm

In [3]:
import heapq
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]

In [4]:
def reconstruct_path(came_from, start, goal):
    current = goal
    path = [current]
    while current != start:
        current = came_from[current]
        path.append(current)
    path.append(start) # optional
    path.reverse() # optional
    return path

In [5]:
def dijkstra_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

### Draw a Maze Table

In [6]:
maze = MazeWithWeights(10, 10)
maze.walls = [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8),
                 (0, 3),(1, 3),(1, 4),(1, 5),(1, 6),(1, 8),(2, 8),(3, 8),(4, 8),(3, 9),(3,3),(5,2),(7,3),(3,4),(4,4),(5,4),(4,5),(4,6),(2,6),(6,5),(6,6),(6,7),(6,8),(6,9)]

s=(0, 9) #start point
g=(9, 0) #goal point
draw_grid(maze, start=s, goal=g)

.  .  .  .  .  .  .  .  .  G  
.  ╋  ╋  ╋  ╋  ╋  ╋  ╋  ╋  .  
.  .  .  .  .  ╋  .  .  ╋  .  
╋  ╋  .  ╋  .  .  .  ╋  ╋  .  
.  ╋  .  ╋  ╋  ╋  .  .  ╋  .  
.  ╋  .  .  ╋  .  ╋  .  ╋  .  
.  ╋  ╋  .  ╋  .  ╋  .  ╋  .  
.  .  .  .  .  .  ╋  .  ╋  .  
.  ╋  ╋  ╋  ╋  .  ╋  .  ╋  .  
S  .  .  ╋  .  .  ╋  .  .  .  


### Do Djkstra Search on the Maze

In [7]:
came_from, cost_so_far = dijkstra_search(maze, s, g)
draw_grid(maze, width=3, point_to=came_from, start=s, goal=g)
print('')
draw_grid(maze, width=3, number=cost_so_far, start=s, goal=g)
print('')
draw_grid(maze, width=3, path=reconstruct_path(came_from, start=s, goal=g))

↓   ←   ←   ←   ←   ←   ←   ←   ←   G   
↓   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   .   
→   →   ↓   ←   ←   ╋   ↓   ←   ╋   .   
╋   ╋   ↓   ╋   ↑   ←   ←   ╋   ╋   .   
↓   ╋   ↓   ╋   ╋   ╋   ↑   ←   ╋   .   
↓   ╋   →   ↓   ╋   ↓   ╋   ↑   ╋   .   
↓   ╋   ╋   ↓   ╋   ↓   ╋   ↑   ╋   .   
↓   ←   ←   ←   ←   ←   ╋   ↑   ╋   .   
↓   ╋   ╋   ╋   ╋   ↑   ╋   ↑   ╋   .   
S   ←   ←   ╋   →   ↑   ╋   ↑   ←   ←   

15  16  17  18  19  20  21  22  23  G   
14  ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   .   
13  12  11  12  13  ╋   17  18  ╋   .   
╋   ╋   10  ╋   14  15  16  ╋   ╋   .   
5   ╋   9   ╋   ╋   ╋   17  18  ╋   .   
4   ╋   8   7   ╋   9   ╋   19  ╋   .   
3   ╋   ╋   6   ╋   8   ╋   20  ╋   .   
2   3   4   5   6   7   ╋   21  ╋   .   
1   ╋   ╋   ╋   ╋   8   ╋   22  ╋   .   
S   1   2   ╋   10  9   ╋   23  24  25  

o   o   o   o   o   o   o   o   o   o   
o   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   .   
o   o   o   .   .   ╋   .   .   ╋   .   
╋   ╋   o   ╋   .   .   .   ╋   ╋   .   
.   ╋   o   ╋ 